In [7]:
import torch
from torchvision import datasets, transforms

In [2]:
class MLP(torch.nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = torch.nn.Linear(28*28, 512)
        self.d1 = torch.nn.Dropout(0.4)
        self.fc2 = torch.nn.Linear(512, 512)
        self.d2 = torch.nn.Dropout(0.4)
        self.fc3 = torch.nn.Linear(512, 10)

    def forward(self, x):
        x = x.view(-1, 28*28)
        x = torch.nn.functional.relu(self.fc1(x))
        x = self.d1(x)
        x = torch.nn.functional.relu(self.fc2(x))
        x = self.d2(x)
        # note that a softmax is not being used explicitly here
        x = self.fc3(x)
        return x

In [3]:
def train(model, device, loader, opt, epoch, log_every=1):
    model.train()
    for batch_idx, (data, target) in enumerate(loader):
        data, target = data.to(device), target.to(device)
        opt.zero_grad()
        output = model(data)
        # Compute the cross entropy loss between input logits and target
        # it means that predictions are not normalized into a probability distribution
        loss = torch.nn.functional.cross_entropy(output, target)
        loss.backward()
        opt.step()
        if batch_idx % log_every == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(loader.dataset),
                100. * batch_idx / len(loader), loss.item()))
            
def test(model, device, loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += torch.nn.functional.cross_entropy(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(loader.dataset),
        100. * correct / len(loader.dataset)))

In [4]:
# define constants (hyperparameters)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
b_size = 64
l_rate = 0.0005
n_epochs = 5

In [5]:
# define the transformations
transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])
# load the MNIST data
train_data = datasets.MNIST('~/.cache/', train=True, download=True, transform=transform)
test_data = datasets.MNIST('~/.cache/', train=False, download=True, transform=transform)
# create data loaders
train_loader = torch.utils.data.DataLoader(train_data, batch_size=b_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=len(test_data), shuffle=False)


In [6]:
model = MLP().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=l_rate)

for epoch in range(n_epochs):
    train(model, device, train_loader, optimizer, epoch, log_every=100)
    test(model, device, test_loader)

Train Epoch: 0 [0/60000 (0%)]	Loss: 2.342268
Train Epoch: 0 [6400/60000 (11%)]	Loss: 0.434161
Train Epoch: 0 [12800/60000 (21%)]	Loss: 0.466048
Train Epoch: 0 [19200/60000 (32%)]	Loss: 0.225441
Train Epoch: 0 [25600/60000 (43%)]	Loss: 0.139917
Train Epoch: 0 [32000/60000 (53%)]	Loss: 0.248043
Train Epoch: 0 [38400/60000 (64%)]	Loss: 0.151754
Train Epoch: 0 [44800/60000 (75%)]	Loss: 0.257226
Train Epoch: 0 [51200/60000 (85%)]	Loss: 0.239137
Train Epoch: 0 [57600/60000 (96%)]	Loss: 0.107415

Test set: Average loss: 0.1224, Accuracy: 9611/10000 (96%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 0.110453
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.174895
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.038679
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.050180
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.124657
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.082266
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.398716
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.107789
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.052293
T